In [5]:
!pip install feedparser
!pip install owlready2
!pip install spade

In [16]:
from IPython.display import display, HTML

html_content = """
<style>
    .title {
        font-size: 24px;
        font-weight: bold;
        color: #00aaff;
        text-align: center;
        margin-bottom: 10px;
    }
    .section {
        font-size: 18px;
        font-weight: bold;
        color: #000;
        background-color: #88ccff;
        padding: 5px;
        border-radius: 5px;
        margin-top: 10px;
    }
    .content {
        font-size: 16px;
        color: #000;
        line-height: 1.5;
        background-color: #eef;
        padding: 5px;
        border-radius: 5px;
    }
    .code {
        background-color: #222;
        padding: 10px;
        border-radius: 5px;
        font-family: monospace;
        color: #00ff00;
    }
</style>

<div class='title'>🕵️‍♂️ EJEMPLO 1: Agente 1 como Coordinador de Noticias 📰</div>

<div class='section'>🔍 ¿Qué hace este código?</div>
<div class='content'>
    Este código implementa un sistema multiagente en Python con SPADE donde <b>el Agente 1</b> actúa como un coordinador que solicita información de noticias a otros agentes (<b>Agente 2, Agente 3 y Agente 4</b>), los cuales consultan diferentes fuentes RSS.<br><br>
    <b>Flujo del programa:</b>
    <ul>
        <li>El usuario ingresa un término de búsqueda.</li>
        <li>El Agente 1 envía la solicitud a los otros tres agentes.</li>
        <li>Cada agente busca coincidencias en su respectivo feed RSS.</li>
        <li>Si encuentran noticias relevantes, las envían de vuelta al Agente 1.</li>
        <li>El Agente 1 recopila y muestra un resumen con los resultados obtenidos.</li>
    </ul>
</div>

<div class='section'>📌 Notas</div>
<div class='content'>
    - <b>El Agente 1 sigue funcionando indefinidamente</b>, solicitando más búsquedas hasta que se detenga manualmente. <br/>
    - Si ningún agente encuentra coincidencias, simplemente se muestra un mensaje indicando que no hubo resultados. <br/>
    - Se puede extender este sistema para incluir más fuentes de noticias o mejorar la ontología utilizada.
</div>
"""

display(HTML(html_content))


In [ ]:
import asyncio
import spade
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour
from spade.template import Template
from spade.message import Message
from owlready2 import get_ontology
import feedparser

# ---------------------------
# Agente 1: Coordinador e interfaz (solicita y resume resultados de 3 agentes)
# ---------------------------
class Agent1(Agent):
    async def setup(self):
        print(f"Agent1 iniciado: {self.jid}")
        self.add_behaviour(SearchAndCoordinateBehaviour())

class SearchAndCoordinateBehaviour(CyclicBehaviour):
    async def run(self):
        search_term = input("Ingrese la cadena de búsqueda de noticias: ")
        
        recipients = [
            "agent2_brivaro@gtirouter.dsic.upv.es",
            "agent3_brivaro@gtirouter.dsic.upv.es",
            "agent4_brivaro@gtirouter.dsic.upv.es" 
        ]
        
        responses = []
        
        for recipient in recipients:
            msg = Message(to=recipient)
            msg.set_metadata("performative", "request")
            msg.body = search_term
            await self.send(msg)
            print(f"Solicitud enviada a {recipient}.")

        # Esperar respuestas de cada agente (timeout 15 segundos)
        for _ in recipients:
            response = await self.receive(timeout=15)
            if response:
                responses.append((str(response.sender), response.body))
                print(f"Respuesta recibida de {response.sender}:")
                print(response.body)
            else:
                print("No se recibió respuesta en el tiempo esperado.")

        # Resumir y mostrar todas las respuestas al usuario
        print("\n--- Resumen de noticias de todos los agentes ---")
        for sender, resp in responses:
            print(f"Desde {sender}:")
            print(resp)
            print("------------------------------------------------")
        print("\n")

        # Si se quiere permitir otra búsqueda, se puede comentar la siguiente línea:
        # self.kill()  # Termina el comportamiento para detener el agente

# ---------------------------
# Agente base para buscadores que consulta un feed RSS y actualiza la ontología rNews
# ---------------------------
class BaseRSSAgent(Agent):
    def __init__(self, jid, password, feed_url, ontology_url, *args, **kwargs):
        super().__init__(jid, password, *args, **kwargs)
        self.feed_url = feed_url
        self.ontology_url = ontology_url

    async def setup(self):
        print(f"{self.__class__.__name__} iniciado: {self.jid}")
        try:
            self.onto = get_ontology(self.ontology_url).load()
            print(f"{self.__class__.__name__}: Ontología rNews cargada.")
        except Exception as e:
            print(f"{self.__class__.__name__}: Error al cargar la ontología:", e)
        
        # Registrar el comportamiento con un template que filtre mensajes con performative "request"
        template = Template(metadata={"performative": "request"})
        self.add_behaviour(RSSSearchBehaviour(self.feed_url), template)

# ---------------------------
# Comportamiento común para agentes que consultan un feed RSS
# ---------------------------
class RSSSearchBehaviour(CyclicBehaviour):
    def __init__(self, feed_url, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.feed_url = feed_url

    async def run(self):
        msg = await self.receive(timeout=10)
        if msg and msg.get_metadata("performative") == "request":
            search_term = msg.body
            print(f"{self.agent.__class__.__name__}: Solicitud para buscar '{search_term}' recibida de {msg.sender}")

            feed = feedparser.parse(self.feed_url)
            matching_entries = []
            for entry in feed.entries:
                # Buscar en el título o resumen (si existe)
                summary = entry.get("summary", "")
                if search_term.lower() in entry.title.lower() or search_term.lower() in summary.lower():
                    matching_entries.append(entry)

            result_text = f"Se encontraron {len(matching_entries)} noticias para '{search_term}':\n"
            for entry in matching_entries:
                result_text += f"- {entry.title}\n"

            # Preparar y enviar la respuesta
            reply = msg.make_reply()
            reply.set_metadata("performative", "inform")
            reply.body = result_text
            await self.send(reply)
            print(f"{self.agent.__class__.__name__}: Respuesta enviada al solicitante.")
        await asyncio.sleep(1)

# ---------------------------
# Agentes consultores (cada uno con un feed RSS distinto)
# ---------------------------
class Agent2(BaseRSSAgent):
    pass  # Usa la implementación base; su feed se definirá al instanciar

class Agent3(BaseRSSAgent):
    pass

class Agent4(BaseRSSAgent):
    pass


In [ ]:
# ---------------------------
# Función principal para iniciar los agentes
# ---------------------------
password = "SPADE"

ontology_url = "http://dev.iptc.org/files/rNews/rnews_1.0_draft3_rdfxml.owl"

feed_url_agent2 = "http://feeds.bbci.co.uk/news/rss.xml"   # BBC News
feed_url_agent3 = "https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml"  # NYTimes
feed_url_agent4 = "https://www.cnet.com/rss/news/"  # CNET News

agent1_jid = "agent1_brivaro@gtirouter.dsic.upv.es"
agent2_jid = "agent2_brivaro@gtirouter.dsic.upv.es"
agent3_jid = "agent3_brivaro@gtirouter.dsic.upv.es"
agent4_jid = "agent4_brivaro@gtirouter.dsic.upv.es"

agent1 = Agent1(agent1_jid, password)
agent2 = Agent2(agent2_jid, password, feed_url_agent2, ontology_url)
agent3 = Agent3(agent3_jid, password, feed_url_agent3, ontology_url)
agent4 = Agent4(agent4_jid, password, feed_url_agent4, ontology_url)

# Iniciar los agentes con auto-registro en el servidor XMPP
await agent2.start(auto_register=True)
await agent3.start(auto_register=True)
await agent4.start(auto_register=True)
await asyncio.sleep(3) # Esperar un poco para asegurar que los agentes están listos
await agent1.start(auto_register=True)

# Esperar a que Agent1 termine su comportamiento (la búsqueda)
while agent1.is_alive():
    await asyncio.sleep(1)

await agent1.stop()
await agent2.stop()
await agent3.stop()
await agent4.stop()
print("Todos los agentes han sido detenidos.")

Agent2 iniciado: agent2_brivaro@gtirouter.dsic.upv.es
Agent2: Ontología rNews cargada.
Agent3 iniciado: agent3_brivaro@gtirouter.dsic.upv.es
Agent3: Ontología rNews cargada.
Agent4 iniciado: agent4_brivaro@gtirouter.dsic.upv.es
Agent4: Ontología rNews cargada.
Agent1 iniciado: agent1_brivaro@gtirouter.dsic.upv.es
Solicitud enviada a agent2_brivaro@gtirouter.dsic.upv.es.
Solicitud enviada a agent3_brivaro@gtirouter.dsic.upv.es.
Solicitud enviada a agent4_brivaro@gtirouter.dsic.upv.es.
Agent2: Solicitud para buscar 'survivors' recibida de agent1_brivaro@gtirouter.dsic.upv.es
Agent2: Respuesta enviada al solicitante.
Agent3: Solicitud para buscar 'survivors' recibida de agent1_brivaro@gtirouter.dsic.upv.es
Agent3: Respuesta enviada al solicitante.
Agent4: Solicitud para buscar 'survivors' recibida de agent1_brivaro@gtirouter.dsic.upv.es
Agent4: Respuesta enviada al solicitante.
Respuesta recibida de agent2_brivaro@gtirouter.dsic.upv.es:
Se encontraron 1 noticias para 'survivors':
- Myanma

In [17]:
html_content = """
<style>
    .title {
        font-size: 24px;
        font-weight: bold;
        color: #00aaff;
        text-align: center;
        margin-bottom: 10px;
    }
    .section {
        font-size: 18px;
        font-weight: bold;
        color: #000;
        background-color: #88ccff;
        padding: 5px;
        border-radius: 5px;
        margin-top: 10px;
    }
    .content {
        font-size: 16px;
        color: #000;
        line-height: 1.5;
        background-color: #eef;
        padding: 5px;
        border-radius: 5px;
    }
    .code {
        background-color: #222;
        padding: 10px;
        border-radius: 5px;
        font-family: monospace;
        color: #00ff00;
    }
</style>

<div class='title'>🕵️‍♂️ EJEMPLO 2: Agente 1 como Coordinador de Noticias 📰</div>

<div class='section'>🔍 ¿Qué hace este código?</div>
<div class='content'>
    Este código implementa un sistema multiagente en Python con SPADE donde <b>el Agente 1</b> actúa como un coordinador que solicita información de noticias a otros agentes (<b>Agente 2, Agente 3 y Agente 4</b>), los cuales consultan diferentes fuentes RSS.<br><br>
    <b>Flujo del programa:</b>
    <ul>
        <li>El usuario ingresa un término de búsqueda.</li>
        <li>El Agente 1 envía la solicitud a los otros tres agentes.</li>
        <li>Cada agente busca coincidencias en su respectivo feed RSS.</li>
        <li>Si encuentran noticias relevantes, las envían de vuelta al Agente 1.</li>
        <li>El Agente 1 recopila y muestra un resumen con los resultados obtenidos.</li>
        <li>Después de 1 minuto, todos los agentes se detienen automáticamente.</li>
    </ul>
</div>

<div class='section'>📌 Notas</div>
<div class='content'>
    - <b>Los agentes se detienen automáticamente</b> después de 1 minuto. <br/>
    - Si ningún agente encuentra coincidencias, simplemente se muestra un mensaje indicando que no hubo resultados. <br/>
    - Se puede extender este sistema para incluir más fuentes de noticias o mejorar la ontología utilizada.
</div>
"""

display(HTML(html_content))

In [ ]:
# ---------------------------
# Agente 1: Coordinador e interfaz (solicita y resume resultados de 3 agentes)
# ---------------------------
class Agent1(Agent):
    async def setup(self):
        print(f"Agent1 iniciado: {self.jid}")
        self.add_behaviour(SearchAndCoordinateBehaviour())

class SearchAndCoordinateBehaviour(CyclicBehaviour):
    async def run(self):
        search_term = input("Ingrese la cadena de búsqueda de noticias: ")
        
        recipients = [
            "agent2_brivaro@gtirouter.dsic.upv.es",
            "agent3_brivaro@gtirouter.dsic.upv.es",
            "agent4_brivaro@gtirouter.dsic.upv.es" 
        ]
        
        responses = []
        
        for recipient in recipients:
            msg = Message(to=recipient)
            msg.set_metadata("performative", "request")
            msg.body = search_term
            await self.send(msg)
            print(f"Solicitud enviada a {recipient}.")

        # Esperar respuestas de cada agente (timeout 15 segundos)
        for _ in recipients:
            response = await self.receive(timeout=15)
            if response:
                responses.append((str(response.sender), response.body))
                print(f"Respuesta recibida de {response.sender}:")
                print(response.body)
            else:
                print("No se recibió respuesta en el tiempo esperado.")

        print("\n--- Resumen de noticias de todos los agentes ---")
        for sender, resp in responses:
            print(f"Desde {sender}:")
            print(resp)
            print("------------------------------------------------")
        print("\n")

        self.kill()

    async def on_end(self):
        print(f"Me las piro, vampiro :) soy el agente con JID: {self.agent.name}, y he terminado mi trabajo.")

# ---------------------------
# Agente base para buscadores que consulta un feed RSS y actualiza la ontología rNews
# ---------------------------
class BaseRSSAgent(Agent):
    def __init__(self, jid, password, feed_url, ontology_url, *args, **kwargs):
        super().__init__(jid, password, *args, **kwargs)
        self.feed_url = feed_url
        self.ontology_url = ontology_url

    async def setup(self):
        print(f"{self.__class__.__name__} iniciado: {self.jid}")
        try:
            self.onto = get_ontology(self.ontology_url).load()
            print(f"{self.__class__.__name__}: Ontología rNews cargada.")
        except Exception as e:
            print(f"{self.__class__.__name__}: Error al cargar la ontología:", e)
        
        # Registrar el comportamiento con un template que filtre mensajes con performative "request"
        template = Template(metadata={"performative": "request"})
        self.add_behaviour(RSSSearchBehaviour(self.feed_url), template)

# ---------------------------
# Comportamiento común para agentes que consultan un feed RSS
# ---------------------------
class RSSSearchBehaviour(CyclicBehaviour):
    def __init__(self, feed_url, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.feed_url = feed_url

    async def run(self):
        msg = await self.receive(timeout=10)
        if msg and msg.get_metadata("performative") == "request":
            search_term = msg.body
            print(f"{self.agent.__class__.__name__}: Solicitud para buscar '{search_term}' recibida de {msg.sender}")

            feed = feedparser.parse(self.feed_url)
            matching_entries = []
            for entry in feed.entries:
                # Buscar en el título o resumen (si existe)
                summary = entry.get("summary", "")
                if search_term.lower() in entry.title.lower() or search_term.lower() in summary.lower():
                    matching_entries.append(entry)

            result_text = f"Se encontraron {len(matching_entries)} noticias para '{search_term}':\n"
            for entry in matching_entries:
                result_text += f"- {entry.title}\n"

            # Preparar y enviar la respuesta
            reply = msg.make_reply()
            reply.set_metadata("performative", "inform")
            reply.body = result_text
            await self.send(reply)
            print(f"{self.agent.__class__.__name__}: Respuesta enviada al solicitante.")
        await asyncio.sleep(1)

# ---------------------------
# Agentes consultores (cada uno con un feed RSS distinto)
# ---------------------------
class Agent2(BaseRSSAgent):
    pass  # Usa la implementación base; su feed se definirá al instanciar

class Agent3(BaseRSSAgent):
    pass

class Agent4(BaseRSSAgent):
    pass


In [ ]:
# ---------------------------
# Función principal para iniciar los agentes
# ---------------------------
password = "SPADE"

ontology_url = "http://dev.iptc.org/files/rNews/rnews_1.0_draft3_rdfxml.owl"

feed_url_agent2 = "http://feeds.bbci.co.uk/news/rss.xml"   # BBC News
feed_url_agent3 = "https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml"  # NYTimes
feed_url_agent4 = "https://www.cnet.com/rss/news/"  # CNET News

agent1_jid = "agent1_brivaro@gtirouter.dsic.upv.es"
agent2_jid = "agent2_brivaro@gtirouter.dsic.upv.es"
agent3_jid = "agent3_brivaro@gtirouter.dsic.upv.es"
agent4_jid = "agent4_brivaro@gtirouter.dsic.upv.es"

agent1 = Agent1(agent1_jid, password)
agent2 = Agent2(agent2_jid, password, feed_url_agent2, ontology_url)
agent3 = Agent3(agent3_jid, password, feed_url_agent3, ontology_url)
agent4 = Agent4(agent4_jid, password, feed_url_agent4, ontology_url)

await agent2.start(auto_register=True)
await agent3.start(auto_register=True)
await agent4.start(auto_register=True)
await asyncio.sleep(3) # Esperar un poco para asegurar que los agentes están listos
await agent1.start(auto_register=True)

print("Los agentes han comenzado. Se detendrán en 1 minuto...")

await asyncio.sleep(60)

print("Deteniendo los agentes...")
await agent1.stop()
await agent2.stop()
await agent3.stop()
await agent4.stop()
print("Agentes detenidos.")

Agent2 iniciado: agent2_brivaro@gtirouter.dsic.upv.es
Agent2: Ontología rNews cargada.
Agent3 iniciado: agent3_brivaro@gtirouter.dsic.upv.es
Agent3: Ontología rNews cargada.
Agent4 iniciado: agent4_brivaro@gtirouter.dsic.upv.es
Agent4: Ontología rNews cargada.
Agent1 iniciado: agent1_brivaro@gtirouter.dsic.upv.es
Los agentes han comenzado. Se detendrán en 1 minuto...
Solicitud enviada a agent2_brivaro@gtirouter.dsic.upv.es.
Solicitud enviada a agent3_brivaro@gtirouter.dsic.upv.es.
Solicitud enviada a agent4_brivaro@gtirouter.dsic.upv.es.
Agent2: Solicitud para buscar 'survivors' recibida de agent1_brivaro@gtirouter.dsic.upv.es
Agent2: Respuesta enviada al solicitante.
Agent3: Solicitud para buscar 'survivors' recibida de agent1_brivaro@gtirouter.dsic.upv.es
Agent3: Respuesta enviada al solicitante.
Agent4: Solicitud para buscar 'survivors' recibida de agent1_brivaro@gtirouter.dsic.upv.es
Agent4: Respuesta enviada al solicitante.
Respuesta recibida de agent2_brivaro@gtirouter.dsic.upv.e